In [36]:
packages <- c("survRM2", "mvtnorm", "ggplot2", "MASS", "tidyr", "survival", "nph",
              "foreach", "doParallel", "cowplot", "IRdisplay", "rlang","tidyverse")

install_and_load <- function(package) {
  if (!require(package, character.only = TRUE)) {
    install.packages(package)
    library(package, character.only = TRUE)
  }
}

invisible(lapply(packages, install_and_load))

n_cores <- detectCores()
cluster <- makeCluster(16)  
registerDoParallel(cluster)
# multi thread
invisible(clusterEvalQ(cluster, #import packages to parallel 
  {
  library('survRM2')
  library('mvtnorm')
  library('cubature')
  library('survival')
  library('nph')
  library("simtrial")
  library("foreach")
  }))

source('Rfiles/3m_comparison.R')
source('Rfiles/Function.R')
clusterExport(cluster, "expo_gen_2stages")

# Optimal Design (minimize E(N))

#### Our method need a given overall sample size N for parameter searching. So we only discuss the optimal design that minimize the expected sample size.
```math
 \bar{PET} = (PET0\ +\ PET1)\ /\ 2  \\
E_N(n) = \bar{PET} * n + (1\ -\ \bar{PET}) * N
```
#### n is the interim sample size of two groups
#### We would like to find a proper n to min(E(N)) under a given ($\alpha$, power and N)

In [37]:
set.seed(2024)
sim_size <- 5000
N <- 200
n <- ceiling( N / 2 )  # total sample size per arm
r <- 90
acc_time <- N / r
cen_time <- 1
lambda_H0 <- 1.2
lambda_H1 <- 0.83
alpha <- 0.05
power <- 0.8

# give a set of optional interim periods
int_time <- seq(0.5, 0.8, by = 4 / N)  # Each time interim sample size increase by 6
interim_list <- int_time * N / r

data_C <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, dist = 'exp', 
                            cen_time = cen_time, arm = 0, interim = interim_list)    
data_E_H0 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H0, dist = 'exp', 
                            cen_time = cen_time, arm = 1, interim = interim_list) 
data_E_H1 <- expo_gen_2stages(N = n * sim_size, acc_time = acc_time, lambda = lambda_H1, dist = 'exp', 
                            cen_time = cen_time, arm = 1, interim = interim_list)

all_result <- data.frame()
for (i in 1 : length(interim_list))
{  
    interim <- interim_list[i]
    rmst_h0_int <- RMST_sim_cal(n = n,data_E = data_E_H0[ , c(2,3,1), i], 
                                data_C = data_C[ , c(2,3,1), i],tau = interim, sim_size = sim_size)
    rmst_h1_int <- RMST_sim_cal(n = n,data_E = data_E_H1[ , c(2,3,1), i], 
                                data_C = data_C[ , c(2,3,1), i],tau = interim, sim_size = sim_size)
    rmst_h0_fin <- RMST_sim_cal(n = n,data_E = data_E_H0[ , c(4,5,1), i], 
                                data_C = data_C[ , c(4,5,1), i],tau = acc_time + cen_time,sim_size = sim_size)
    rmst_h1_fin <- RMST_sim_cal(n = n,data_E = data_E_H1[ , c(4,5,1), i], 
                                data_C = data_C[ , c(4,5,1), i],tau = acc_time + cen_time,sim_size = sim_size)
    rmst_data <- rbind(rmst_h0_int, rmst_h1_int, rmst_h0_fin, rmst_h1_fin)

    mu_cov_h0 <- mu_cov_mc(rmst_int = rmst_h0_int, rmst_fin = rmst_h0_fin, sim_size = sim_size)
    mu_cov_h1 <- mu_cov_mc(rmst_int = rmst_h1_int, rmst_fin = rmst_h1_fin, sim_size = sim_size)

    best_our <- adp_grid_src(rmst_data = rmst_data, mu_cov_h0 = mu_cov_h0, mu_cov_h1 = mu_cov_h1, 
                int_n = interim * r, fin_n = 2 * n, sim_size = sim_size, type = 'Complex',
                alpha = alpha, power = power)
    best_our$interim_n <- interim * r

    all_result <- rbind(all_result, best_our)
}


In [38]:
all_result

m1,t1,m2,t2,lambda,gamma,PET0,PET1,alpha,power,EN,interim_n
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0.000000e+00,0.0000000,0.0000000,0.0000000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,NA,100
0.000000e+00,0.0000000,0.0000000,0.0000000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,NA,104
0.000000e+00,0.0000000,0.0000000,0.0000000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,NA,108
0.000000e+00,0.0000000,0.0000000,0.0000000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,NA,112
0.000000e+00,0.0000000,0.0000000,0.0000000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,NA,116
0.000000e+00,0.0000000,0.0000000,0.0000000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,NA,120
0.000000e+00,0.0000000,0.0000000,0.0000000,0.00,0.00,0.0000,0.0000,0.0000,0.0000,NA,124
1.791735e-03,0.6493946,0.1515372,0.9196343,0.99,0.16,0.6330,0.0900,0.0250,0.8030,173.9720,128
7.138312e-03,0.6602750,0.1510168,0.9184726,0.99,0.16,0.6582,0.0974,0.0250,0.8006,174.3096,132
